<a href="https://colab.research.google.com/github/ishaanalmeida/stat-arb-mean-reversion/blob/main/stat_arb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Statistical Arbitrage: Mean Reversion in Equity Pairs

In [1]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
!git clone https://github.com/ishaanalmeida/stat-arb-mean-reversion.git

Cloning into 'stat-arb-mean-reversion'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


In [3]:
%cd stat-arb-mean-reversion

/content/stat-arb-mean-reversion


# 1. Project Overview and Hypothesis

# 2. Data Collection and Preprocessing

# 3. Cointegration Testing (ADF + Johansen)

# 4. Z-score Calculation and Entry/Exit Logic

# 5. Backtesting the Strategy

# 6. Performance Evaluation (PnL, Sharpe, Drawdown)


# 7. Visualization

# 8. Observations and Next Steps